In [ ]:
!pkill -f streamlit
from pyngrok import ngrok
ngrok.kill()

ModuleNotFoundError: No module named 'pyngrok'

In [1]:
# ===================== INSTALL REQUIRED PACKAGES =====================
!pip install fastapi tsfresh uvicorn pyngrok pandas numpy prophet scikit-learn tsfresh matplotlib plotly streamlit requests  typing Optional reportlab --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.6 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("38Zld081miKTF2WqFshAeRp1WhY_88mABk46Ct91p7asFaLZH")
# ngrok.set_auth_token("380ZzaoDdXGvfjMya7VVYPbpJA4_3dcMEvWNppZQDdUGs9j7p")

In [3]:
%%writefile backend.py
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse, FileResponse
from fastapi.middleware.cors import CORSMiddleware
import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import numpy as np
import os
from reportlab.platypus import (
    SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
)
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from datetime import datetime
import glob
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from openai import OpenAI
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.feature_selection import VarianceThreshold
from fastapi import HTTPException
import traceback

from fastapi import Body
from typing import Optional




# ================= APP =================
app = FastAPI(title="FitPulse Backend")

# ================= CORS =================
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ================= GLOBAL STATE =================
CLEAN_DF = None
FEATURE_DF = None
ANOMALY_DF = None
RULE_RECOMMENDATIONS = None

# ================= ROOT =================
def classify_severity(count: int):
    if count >= 20:
        return "High"
    elif count >= 5:
        return "Medium"
    return "Low"

def df_to_table(df, max_rows=10):
    data = [df.columns.tolist()] + df.head(max_rows).values.tolist()
    return Table(data, repeatRows=1)


@app.get("/")
def root():
    return {"message": "FitPulse Backend is running"}

# ================= PREPROCESSING (CSV + JSON) =================
@app.post("/preprocess")
async def preprocess(file: UploadFile = File(...)):
    global CLEAN_DF

    try:
        if file.filename.endswith(".csv"):
            df = pd.read_csv(file.file)
        elif file.filename.endswith(".json"):
            df = pd.read_json(file.file)
        else:
            return JSONResponse(status_code=400, content={"error": "Only CSV or JSON supported"})
    except Exception as e:
        return JSONResponse(status_code=400, content={"error": f"Invalid file: {e}"})

    required_cols = ["user_id", "date", "TotalSteps", "avg_heart_rate", "total_sleep_minutes"]
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        return JSONResponse(status_code=400, content={"error": f"Missing columns: {missing}"})

    # ---------- CLEANING ----------
    # ---------- CLEANING ----------
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    for col in ["TotalSteps", "avg_heart_rate", "total_sleep_minutes"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    df["TotalSteps"].fillna(0, inplace=True)
    df["avg_heart_rate"].fillna(df["avg_heart_rate"].median(), inplace=True)

    # ✅ SAFE sleep handling
    df["total_sleep_minutes"].fillna(0, inplace=True)

    # ---------- AGGREGATE (FIXED) ----------
    df = df.groupby(["user_id", "date"], as_index=False).agg({
        "TotalSteps": "sum",
        "avg_heart_rate": "mean",
        "total_sleep_minutes": "sum"   # ✅ SUM, NOT MEAN
    })


    # ---------- RENAME FOR UI ----------
    df.rename(columns={
        "avg_heart_rate": "heart_rate",
        "TotalSteps": "steps",
        "total_sleep_minutes": "sleep"
    }, inplace=True)

    # Convert sleep minutes → hours (matches screenshot ~7.05)
    df["sleep"] = (df["sleep"] / 60).round(2)

    CLEAN_DF = df
    df.to_csv("clean_data.csv", index=False)


    return {
        "status": "success",
        "overview": {
            "rows_loaded": len(df),
            "users": df["user_id"].nunique(),
            "days": df["date"].nunique(),
            "avg_hr": round(df["heart_rate"].mean(), 1)
        },
        "preview": df.to_dict(orient="records")
    }



# ================= OVERVIEW =================
@app.get("/overview")
def overview():
    if CLEAN_DF is None:
        return {"error": "Run /preprocess first"}
    df = CLEAN_DF
    return {
        "rows": len(df),
        "users_count": df["user_id"].nunique(),
        "users_list": df["user_id"].astype(str).unique().tolist(),  # <-- add this
        "start_date": str(df["date"].min().date()),
        "end_date": str(df["date"].max().date()),
        "avg_heart_rate": round(df["heart_rate"].mean(), 2),
        "avg_steps": round(df["steps"].mean(), 2),
        "avg_sleep_hours": round(df["sleep"].mean(), 2),
    }
@app.get("/dataframe")
def get_clean_dataframe():
    if CLEAN_DF is None or CLEAN_DF.empty:
        return {"rows": []}

    df = CLEAN_DF.copy()
    df["date"] = df["date"].astype(str)
    df["user_id"] = df["user_id"].astype(str)

    return {"rows": df.to_dict(orient="records")}

# ================= FEATURES + ANOMALIES =================
@app.post("/module2")
def module2():
    global FEATURE_DF, ANOMALY_DF, RULE_RECOMMENDATIONS
    if CLEAN_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run /preprocess first"})

    df = CLEAN_DF.copy().sort_values("date")

    # ================= TSFRESH FEATURES =================
    # ================= TSFRESH FEATURES =================
    try:
        ts_df = df.melt(
            id_vars=["user_id", "date"],
            value_vars=["heart_rate", "steps", "sleep"],
            var_name="metric",
            value_name="value"
        )

        ts_features = extract_features(
            ts_df,
            column_id="user_id",
            column_sort="date",
            column_kind="metric",
            column_value="value",
            disable_progressbar=True
        )

        # Impute missing values
        impute(ts_features)

        # Feature selection (variance threshold)
        selector = VarianceThreshold(threshold=0.01)
        ts_features_selected = selector.fit_transform(ts_features)

        TSFRESH_DF = pd.DataFrame(
            ts_features_selected,
            index=ts_features.index,
            columns=ts_features.columns[selector.get_support()]
        )

        TSFRESH_DF.index.name = "user_id"
        TSFRESH_DF.reset_index(inplace=True)

        # Save full TSFRESH features
        TSFRESH_DF.to_csv("tsfresh_features.csv", index=False)

    except Exception:
        print("TSFRESH extraction failed\n", traceback.format_exc())


    FEATURE_DF = df.copy()

    # Rolling 7-day mean features
    for col in ["steps", "heart_rate", "sleep"]:
        FEATURE_DF[f"{col}_mean_7"] = FEATURE_DF[col].rolling(7).mean()
        FEATURE_DF[f"{col}_std_7"] = FEATURE_DF[col].rolling(7).std()
        FEATURE_DF[f"{col}_skew_7"] = FEATURE_DF[col].rolling(7).skew()
        FEATURE_DF[f"{col}_kurt_7"] = FEATURE_DF[col].rolling(7).kurt()

    FEATURE_DF.to_csv("feature_data.csv", index=False)

    # Rule-based anomalies
    records = []
    for _, r in df.iterrows():
        if r["heart_rate"] > 120: records.append((r["user_id"], r["date"], "heart_rate_high"))
        if r["heart_rate"] < 40: records.append((r["user_id"], r["date"], "heart_rate_low"))
        if r["steps"] == 0: records.append((r["user_id"], r["date"], "no_steps"))
        # ✅ Improved sleep anomaly logic
        if r["sleep"] == 0:
            continue  # no sleep data → skip
        elif r["sleep"] < 3:
            records.append((r["user_id"], r["date"], "sleep_low"))
        elif r["sleep"] > 10:
            records.append((r["user_id"], r["date"], "sleep_high"))

    rule_df = pd.DataFrame(records, columns=["user_id", "date", "metric"])

    # DBSCAN anomalies
    X_scaled = StandardScaler().fit_transform(df[["heart_rate", "steps", "sleep"]])
    labels = DBSCAN(eps=1.2, min_samples=3).fit_predict(X_scaled)
    df["cluster"] = labels
    dbscan_df = df[df["cluster"] == -1][["user_id", "date"]].copy()
    dbscan_df["metric"] = "dbscan_outlier"

    ANOMALY_DF = pd.concat([rule_df, dbscan_df], ignore_index=True)

    summary_df = (
        ANOMALY_DF
        .groupby(["user_id", "metric"])
        .size()
        .reset_index(name="count")
    )

    summary_df["severity"] = summary_df["count"].apply(classify_severity)

    # Save both versions
    ANOMALY_DF.to_csv("anomaly_raw.csv", index=False)
    summary_df.to_csv("anomaly_report.csv", index=False)


    recs = []

    for _, row in summary_df.iterrows():
        user = row["user_id"]
        metric = row["metric"]
        severity = row["severity"]

        if metric == "heart_rate_high":
            recs.append({
                "user_id": user,
                "issue": "High heart rate",
                "severity": severity,
                "recommendation": "Reduce high-intensity workouts and consult a physician if persistent."
            })

        elif metric == "heart_rate_low":
            recs.append({
                "user_id": user,
                "issue": "Low heart rate",
                "severity": severity,
                "recommendation": "Ensure adequate nutrition and consult a healthcare professional."
            })

        elif metric == "no_steps":
            recs.append({
                "user_id": user,
                "issue": "No physical activity",
                "severity": severity,
                "recommendation": "Increase daily movement with light walks or stretching."
            })

        elif metric == "sleep_low":
            recs.append({
                "user_id": user,
                "issue": "Low sleep duration",
                "severity": severity,
                "recommendation": "Aim for at least 7–8 hours of consistent sleep."
            })

        elif metric == "sleep_high":
            recs.append({
                "user_id": user,
                "issue": "Excessive sleep duration",
                "severity": severity,
                "recommendation": "Excess sleep may indicate fatigue or health issues."
            })

        elif metric == "dbscan_outlier":
            recs.append({
                "user_id": user,
                "issue": "Unusual health pattern",
                "severity": severity,
                "recommendation": "Monitor trends closely; consider lifestyle adjustments."
            })

    RULE_RECOMMENDATIONS = pd.DataFrame(recs)
    RULE_RECOMMENDATIONS.to_csv("recommendations.csv", index=False)

    return {
        "status": "success",
        "total_anomalies": len(ANOMALY_DF),
        "summary_rows": len(summary_df)
    }
@app.get("/module3/feature-table")
def feature_table():
    global FEATURE_DF

    if FEATURE_DF is None or FEATURE_DF.empty:
        return {"rows": []}

    df = FEATURE_DF.copy()

    # ✅ 1. Convert datetime → string
    if "date" in df.columns:
        df["date"] = df["date"].astype(str)

    # ✅ 2. Replace NaN / inf with None (JSON-safe)
    df = df.replace([np.nan, np.inf, -np.inf], None)

    # ✅ 3. Convert numpy types → Python native
    df = df.astype(object)

    rows = df.head(200).to_dict(orient="records")

    return {"rows": rows}




# ================= ANOMALY SUMMARY =================

@app.get("/module3/summary")
def anomaly_summary(user_id: str = "All"):
    if ANOMALY_DF is None or ANOMALY_DF.empty:
        return {"summary": {}}

    df = ANOMALY_DF.copy()
    df["user_id"] = df["user_id"].astype(str)

    if user_id != "All":
        df = df[df["user_id"] == str(user_id)]

    summary = (
        df.groupby("metric")
          .size()
          .to_dict()
    )

    return {"summary": summary}

def plot_prophet(df, column, fname, ylabel):
    df2 = df[["date", column]].rename(columns={"date": "ds", column: "y"})

    if len(df2) < 10:
        return None

    model = Prophet()
    model.fit(df2)

    forecast = model.predict(df2)

    df2["yhat"] = forecast["yhat"]
    df2["residual"] = df2["y"] - df2["yhat"]

    threshold = 2.5 * df2["residual"].std()
    outliers = df2[abs(df2["residual"]) > threshold]

    plt.figure(figsize=(12, 5))
    plt.plot(df2["ds"], df2["y"], label=f"Actual {ylabel}", color="blue")
    plt.plot(df2["ds"], df2["yhat"], label="Trend", color="red")

    # ✅ CONFIDENCE INTERVALS
    plt.fill_between(
        df2["ds"],
        forecast["yhat_lower"],
        forecast["yhat_upper"],
        alpha=0.3,
        label="Confidence Interval"
    )

    plt.scatter(outliers["ds"], outliers["y"], color="red", label="Anomaly")

    plt.xlabel("Date")
    plt.ylabel(ylabel)
    plt.title(f"{ylabel} Trend with Prophet")
    plt.legend()
    plt.grid(True)

    plt.savefig(fname)
    plt.close()

    return fname


@app.get("/module3/tsfresh-summary")
def tsfresh_summary():
    try:
        if not os.path.exists("tsfresh_features.csv"):
            return JSONResponse(content={"features": []})

        df = pd.read_csv("tsfresh_features.csv")

        # If user_id exists, drop it for full-dataset summary
        if "user_id" in df.columns:
            df = df.drop(columns=["user_id"])

        # Only numeric features
        numeric_df = df.select_dtypes(include=[np.number])
        if numeric_df.empty:
            return JSONResponse(content={"features": []})

        # Top 10 features by variance
        var_series = numeric_df.var().dropna()
        if var_series.empty:
            return JSONResponse(content={"features": []})

        top_var = var_series.sort_values(ascending=False).head(10)

        feature_list = []
        for feat, var in top_var.items():
            feature_list.append({
                "description": humanize_tsfresh_feature(feat),
                "importance": float(np.log10(var + 1)),
                "feature": feat
            })

        return {"features": feature_list}

    except Exception:
        print("TSFRESH SUMMARY ERROR\n", traceback.format_exc())
        return JSONResponse(content={"features": []})





def humanize_tsfresh_feature(name):
    if "__c3__" in name:
        return "Non-linear step behavior from previous days"
    if "time_reversal_asymmetry" in name:
        return "Irregular activity trend"
    if "abs_energy" in name:
        return "Overall activity energy"
    if "change_quantiles" in name:
        return "Sudden activity changes"
    if "linear_trend" in name:
        return "Long-term activity trend"
    if "lag_" in name:
        return "Previous day effect"
    return "General activity pattern"

@app.get("/module3/health-score")
def health_score(user_id: str = "All"):
    if CLEAN_DF is None or CLEAN_DF.empty:
        return {"error": "No data available. Run /preprocess first."}

    df = CLEAN_DF.copy()
    df["user_id"] = df["user_id"].astype(str)

    if user_id != "All":
        df = df[df["user_id"] == str(user_id)]

    if df.empty:
        return {"error": "No data for selected user."}

    # ---------- Calculate continuous scores ----------
    hr_score = max(0, 100 - (df['heart_rate'] - 70).abs().mean())
    steps_score = min(100, df['steps'].mean() / 10000 * 100)
    sleep_score = 100 - abs(df['sleep'].mean() - 7.5) / 7.5 * 100

    # ---------- Final health score ----------
    health_score = round((hr_score + steps_score + sleep_score) / 3, 1)

    # Optional: classify as High/Medium/Low for donut chart
    if health_score >= 80:
        status = "High"
    elif health_score >= 50:
        status = "Medium"
    else:
        status = "Low"

    # ---------- Return JSON for frontend ----------
    return {
        "user_id": user_id,
        "health_score": health_score,
        "status": status,
        "components": {
            "heart_rate": round(hr_score, 1),
            "steps": round(steps_score, 1),
            "sleep": round(sleep_score, 1)
        }
    }

@app.get("/module3/prophet/{metric}")
def prophet_metric(metric: str, user_id: str = "All"):
    mapping = {
        "heart_rate": "Heart Rate (BPM)",
        "steps": "Steps",
        "sleep": "Sleep (Hours)"
    }

    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run /module2 first"})

    if metric not in mapping:
        return JSONResponse(status_code=400, content={"error": "Invalid metric"})

    # ✅ FORCE STRING TYPES (THIS IS THE KEY FIX)
    df = FEATURE_DF.copy()
    df["user_id"] = df["user_id"].astype(str)
    user_id = str(user_id)

    # ✅ APPLY USER FILTER
    if user_id != "All":
        df = df[df["user_id"] == user_id]

    if df.empty:
        return JSONResponse(status_code=400, content={"error": "No data for selected user"})

    fname = f"prophet_{metric}.png"

    # ✅ PASS FILTERED DATA ONLY
    if plot_prophet(df, metric, fname, mapping[metric]):
        return FileResponse(fname)

    return JSONResponse(status_code=400, content={"error": "Not enough data"})



# ================= DBSCAN VISUALIZATION =================
# ================= DBSCAN VISUALIZATION =================
@app.get("/module3/dbscan")
def dbscan_viz(user_id: str = "All"):
    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run /module2 first"})

    df = FEATURE_DF.copy()
    df["user_id"] = df["user_id"].astype(str)

    if user_id != "All":
        df = df[df["user_id"] == user_id]

    if df.empty:
        return JSONResponse(status_code=400, content={"error": "No data for selected user"})

    X_scaled = StandardScaler().fit_transform(df[["heart_rate", "steps", "sleep"]])
    labels = DBSCAN(eps=1.2, min_samples=3).fit_predict(X_scaled)
    df["cluster"] = labels

    normal = df[df["cluster"] != -1]
    outliers = df[df["cluster"] == -1]

    plt.figure(figsize=(10, 6))
    plt.scatter(normal["steps"], normal["heart_rate"], label="Normal", alpha=0.6)
    plt.scatter(outliers["steps"], outliers["heart_rate"], label="Outlier", alpha=0.9)

    plt.xlabel("Steps")
    plt.ylabel("Heart Rate")
    plt.title("DBSCAN Clustering")
    plt.legend()
    plt.grid(True)

    fname = "dbscan.png"
    plt.savefig(fname)
    plt.close()

    return FileResponse(fname)
@app.get("/module3/pca-clusters")
def pca_clusters():
    if not os.path.exists("behavior_clusters.csv"):
        return JSONResponse(status_code=400, content={"error": "Run /module2 first"})

    df = pd.read_csv("behavior_clusters.csv")

    plt.figure(figsize=(10, 6))
    for c in df["cluster"].unique():
        d = df[df["cluster"] == c]
        plt.scatter(d["pca1"], d["pca2"], label=f"Cluster {c}", alpha=0.7)

    plt.xlabel("PCA Component 1")
    plt.ylabel("PCA Component 2")
    plt.title("Behavioral Clustering (PCA + KMeans)")
    plt.legend()
    plt.grid(True)

    fname = "pca_clusters.png"
    plt.savefig(fname)
    plt.close()

    return FileResponse(fname)

@app.get("/module3/raw-vs-rolling/{metric}")
def raw_vs_rolling(metric: str, user_id: str = "All"):
    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run /module2 first"})

    if metric not in ["heart_rate", "steps", "sleep"]:
        return JSONResponse(status_code=400, content={"error": "Invalid metric"})

    df = FEATURE_DF.copy()
    df["user_id"] = df["user_id"].astype(str)

    if user_id != "All":
        df = df[df["user_id"] == user_id]

    plt.figure(figsize=(12, 5))
    plt.plot(df["date"], df[metric], label="Raw", alpha=0.6)
    plt.plot(df["date"], df[f"{metric}_mean_7"], label="Rolling Mean (7)", linewidth=2)

    plt.title(f"{metric} – Raw vs Rolling Feature")
    plt.xlabel("Date")
    plt.ylabel(metric)
    plt.legend()
    plt.grid(True)

    fname = f"{metric}_raw_vs_rolling.png"
    plt.savefig(fname)
    plt.close()

    return FileResponse(fname)


# ================= DISTRIBUTION =================
# ================= DISTRIBUTION =================
@app.get("/module3/distribution/{metric}")
def distribution(metric: str, user_id: str = "All"):
    if FEATURE_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run /module2 first"})

    if metric not in ["heart_rate", "steps", "sleep"]:
        return JSONResponse(status_code=400, content={"error": "Invalid metric"})

    df = FEATURE_DF.copy()
    df["user_id"] = df["user_id"].astype(str)

    if user_id != "All":
        df = df[df["user_id"] == user_id]

    if df.empty:
        return JSONResponse(status_code=400, content={"error": "No data for selected user"})

    colors_map = {
        "heart_rate": "crimson",
        "steps": "royalblue",
        "sleep": "green"
    }

    plt.figure(figsize=(10, 5))
    plt.hist(df[metric], bins=20, color=colors_map[metric], edgecolor="black")
    plt.xlabel(metric)
    plt.ylabel("Frequency")
    plt.title(f"{metric} Distribution")
    plt.grid(True)

    fname = f"{metric}_dist.png"
    plt.savefig(fname)
    plt.close()

    return FileResponse(fname)



@app.get("/module3/anomaly-with-recommendations")
def anomaly_with_recommendations(user_id: str = "All"):
    global ANOMALY_DF, RULE_RECOMMENDATIONS

    # ------------------ Use real data if available ------------------
    if ANOMALY_DF is not None and not ANOMALY_DF.empty:
        df_anom = ANOMALY_DF.copy()
        df_anom["user_id"] = df_anom["user_id"].astype(str)

        if user_id != "All":
            df_anom = df_anom[df_anom["user_id"] == str(user_id)]

        if df_anom.empty:
            # fallback to dummy
            use_dummy = True
        else:
            use_dummy = False
    else:
        use_dummy = True

    # ------------------ Dummy fallback ------------------
    if use_dummy:
        return {
            "rows": [
                {"date": "2026-01-23", "issue": "High heart rate", "severity": "High",
                 "recommendation": "Reduce high-intensity workouts"},
                {"date": "2026-01-23", "issue": "No physical activity", "severity": "Medium",
                 "recommendation": "Increase daily movement"},
                {"date": "2026-01-23", "issue": "Low sleep duration", "severity": "Low",
                 "recommendation": "Aim for at least 7–8 hours of sleep"}
            ]
        }

    # ------------------ Map metric → issue ------------------
    metric_to_issue = {
        "heart_rate_high": "High heart rate",
        "heart_rate_low": "Low heart rate",
        "no_steps": "No physical activity",
        "sleep_low": "Low sleep duration",
        "sleep_high": "Excessive sleep duration",
        "dbscan_outlier": "Unusual health pattern"
    }
    df_anom["issue"] = df_anom["metric"].map(metric_to_issue)

    # ------------------ Merge with recommendations ------------------
    if RULE_RECOMMENDATIONS is not None and not RULE_RECOMMENDATIONS.empty:
        df_reco = RULE_RECOMMENDATIONS.copy()
        df_reco["user_id"] = df_reco["user_id"].astype(str)

        combined = df_anom.merge(
            df_reco,
            on=["user_id", "issue"],
            how="left"
        )

        combined = combined[["user_id", "date", "issue", "severity", "recommendation"]]


        # Sort by severity
        combined["severity_rank"] = combined["severity"].map({"High": 3, "Medium": 2, "Low": 1})
        combined = combined.sort_values(by="severity_rank", ascending=False).drop(columns="severity_rank")

        return {"rows": combined.to_dict(orient="records")}

    # If no recommendations, return anomalies only
    return {"rows": df_anom.to_dict(orient="records")}
from fastapi.responses import FileResponse
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import os

@app.get("/module4/report")
def generate_user_report(user_id: str):
    global CLEAN_DF, RULE_RECOMMENDATIONS

    # ---------- VALIDATION ----------
    if CLEAN_DF is None or CLEAN_DF.empty:
        return {"error": "No data available"}

    df_user = CLEAN_DF[CLEAN_DF["user_id"].astype(str) == str(user_id)]

    if df_user.empty:
        return {"error": "User not found"}

    # ---------- PATH SETUP ----------
    os.makedirs("reports", exist_ok=True)
    pdf_path = f"reports/user_{user_id}_health_report.pdf"
    donut_path = f"reports/donut_{user_id}.png"

    # ---------- DASHBOARD METRICS ----------
    avg_hr = round(df_user["heart_rate"].mean(), 1)
    avg_steps = round(df_user["steps"].mean(), 0)
    avg_sleep = round(df_user["sleep"].mean(), 1)

    # ---------- HEALTH SCORE CALCULATION ----------
    hr_score = max(0, 100 - abs(avg_hr - 70))
    steps_score = min(100, (avg_steps / 10000) * 100)
    sleep_score = max(0, 100 - abs(avg_sleep - 7.5) * 10)
    health_score = round((hr_score + steps_score + sleep_score) / 3, 1)

    # ---------- CREATE DONUT CHART IMAGE ----------
    plt.figure(figsize=(4, 4))
    plt.pie(
        [hr_score, steps_score, sleep_score],
        labels=["Heart Rate", "Steps", "Sleep"],
        autopct="%1.1f%%",
        startangle=90
    )
    plt.title(f"Health Score: {health_score}%")
    plt.savefig(donut_path)
    plt.close()

    # ---------- PDF CREATION ----------
    styles = getSampleStyleSheet()
    story = []

    # ---------- TITLE ----------
    story.append(Paragraph("<b>User Health Dashboard Report</b>", styles["Title"]))
    story.append(Spacer(1, 12))

    story.append(Paragraph(f"User ID: {user_id}", styles["Normal"]))
    story.append(Paragraph(f"Total Records: {len(df_user)}", styles["Normal"]))
    story.append(Spacer(1, 12))

    # ---------- METRICS SECTION ----------
    story.append(Paragraph("<b>Health Metrics</b>", styles["Heading2"]))
    story.append(Spacer(1, 8))

    story.append(Paragraph(f"Average Heart Rate: {avg_hr} BPM", styles["Normal"]))
    story.append(Paragraph(f"Average Steps: {avg_steps}", styles["Normal"]))
    story.append(Paragraph(f"Average Sleep: {avg_sleep} hours", styles["Normal"]))
    story.append(Spacer(1, 12))

    # ---------- DONUT IMAGE ----------
    story.append(Paragraph("<b>Overall Health Status</b>", styles["Heading2"]))
    story.append(Spacer(1, 10))

    if os.path.exists(donut_path):
        story.append(Image(donut_path, width=200, height=200))
        story.append(Spacer(1, 12))

    # ---------- HEALTH ISSUES ----------
    story.append(Paragraph("<b>Health Issues & Recommendations</b>", styles["Heading2"]))
    story.append(Spacer(1, 8))

    if RULE_RECOMMENDATIONS is not None:
        df_reco = RULE_RECOMMENDATIONS[
            RULE_RECOMMENDATIONS["user_id"].astype(str) == str(user_id)
        ]

        if df_reco.empty:
            story.append(Paragraph("No significant issues detected.", styles["Normal"]))
        else:
            for _, row in df_reco.iterrows():
                story.append(
                    Paragraph(
                        f"- <b>{row['issue']}</b> ({row['severity']}): {row['recommendation']}",
                        styles["Normal"]
                    )
                )

    # ---------- BUILD PDF ----------
    doc = SimpleDocTemplate(pdf_path)
    doc.build(story)

    return FileResponse(
        pdf_path,
        media_type="application/pdf",
        filename=f"user_{user_id}_health_report.pdf"
    )
@app.get("/module4/user-dashboard-report")
def user_dashboard_report(user_id: str):
    if CLEAN_DF is None:
        return {"error": "Run preprocessing first"}

    df = FEATURE_DF.copy()
    df["user_id"] = df["user_id"].astype(str)
    df = df[df["user_id"] == str(user_id)].sort_values("date")

    if df.empty:
        return {"error": "User not found"}

    os.makedirs("reports", exist_ok=True)
    pdf_path = f"reports/user_{user_id}_dashboard.pdf"

    styles = getSampleStyleSheet()
    story = []

    # ---------- TITLE ----------
    story.append(Paragraph("<b>FitPulse – Personalized Health Report</b>", styles["Title"]))
    story.append(Spacer(1, 12))
    story.append(Paragraph(f"User ID: {user_id}", styles["Normal"]))
    story.append(Paragraph(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M')}", styles["Normal"]))
    story.append(Spacer(1, 20))

    # ---------- METRICS ----------
    avg_hr = round(df["heart_rate"].mean(), 1)
    avg_steps = round(df["steps"].mean(), 0)
    avg_sleep = round(df["sleep"].mean(), 1)

    story.append(Paragraph("<b>Health Metrics</b>", styles["Heading2"]))
    story.append(Paragraph(f"Average Heart Rate: {avg_hr} BPM", styles["Normal"]))
    story.append(Paragraph(f"Average Steps: {avg_steps}", styles["Normal"]))
    story.append(Paragraph(f"Average Sleep: {avg_sleep} hours", styles["Normal"]))
    story.append(Spacer(1, 12))

    # ---------- HEALTH SCORE ----------
    hr_score = max(0, 100 - abs(avg_hr - 70))
    steps_score = min(100, (avg_steps / 10000) * 100)
    sleep_score = max(0, 100 - abs(avg_sleep - 7.5) * 10)

    donut_path = f"reports/donut_{user_id}.png"

    plt.figure(figsize=(4, 4))
    plt.pie(
        [hr_score, steps_score, sleep_score],
        labels=["Heart Rate", "Steps", "Sleep"],
        autopct="%1.1f%%",
        startangle=90
    )
    plt.title("Overall Health Score")
    plt.savefig(donut_path)
    plt.close()

    story.append(Paragraph("<b>Health Score Breakdown</b>", styles["Heading2"]))
    story.append(Image(donut_path, width=200, height=200))
    story.append(Spacer(1, 20))

    # ---------- ANOMALIES ----------
    story.append(Paragraph("<b>Detected Issues & Recommendations</b>", styles["Heading2"]))

    if RULE_RECOMMENDATIONS is not None:
        reco = RULE_RECOMMENDATIONS[
            RULE_RECOMMENDATIONS["user_id"].astype(str) == str(user_id)
        ]

        if reco.empty:
            story.append(Paragraph("No significant issues detected.", styles["Normal"]))
        else:
            for _, r in reco.iterrows():
                story.append(Paragraph(
                    f"- <b>{r['issue']}</b> ({r['severity']}): {r['recommendation']}",
                    styles["Normal"]
                ))
    else:
        story.append(Paragraph("No recommendation engine data available.", styles["Normal"]))

    # ---------- RECENT ACTIVITY TABLE (USER-FRIENDLY) ----------
    story.append(Spacer(1, 20))
    story.append(Paragraph("<b>Recent Activity Summary (Last 7 Days)</b>", styles["Heading2"]))

    display_cols = ["date", "steps", "heart_rate", "sleep"]

    table_df = df[display_cols].tail(7).copy()

    # Rename columns for humans
    table_df.rename(columns={
        "date": "Date",
        "steps": "Steps Walked",
        "heart_rate": "Heart Rate (BPM)",
        "sleep": "Sleep (Hours)"
    }, inplace=True)

    # Round values for clean display
    table_df["Sleep (Hours)"] = table_df["Sleep (Hours)"].round(1)

    # Add simple heart rate status (optional but recommended)
    def hr_status(hr):
        if hr < 60:
            return "Low"
        elif hr > 100:
            return "High"
        return "Normal"

    table_df["HR Status"] = table_df["Heart Rate (BPM)"].apply(hr_status)

    story.append(df_to_table(table_df))


    # ---------- RAW vs ROLLING AVERAGE ----------
    metrics = ["heart_rate", "steps", "sleep"]

    for metric in metrics:
        if metric in df.columns:
            df[f"{metric}_rolling"] = df[metric].rolling(7, min_periods=1).mean()

            plt.figure(figsize=(6, 3))
            plt.plot(df["date"], df[metric], label="Raw", alpha=0.7)
            plt.plot(df["date"], df[f"{metric}_rolling"], label="Rolling Avg (7 days)", linewidth=2)
            plt.title(f"{metric.replace('_', ' ').title()} – Raw vs Rolling Average")
            plt.xlabel("Date")
            plt.ylabel(metric.replace("_", " ").title())
            plt.legend()
            plt.tight_layout()

            plot_path = f"reports/{metric}_rolling_{user_id}.png"
            plt.savefig(plot_path)
            plt.close()

            story.append(Spacer(1, 12))
            story.append(Paragraph(
                f"<b>{metric.replace('_', ' ').title()} Trend</b>",
                styles["Heading2"]
            ))
            story.append(Image(plot_path, width=450, height=220))

    # ---------- DBSCAN CLUSTER ----------
    cluster_cols = ["heart_rate", "steps", "sleep"]

    if all(col in df.columns for col in cluster_cols) and len(df) >= 3:
        X = StandardScaler().fit_transform(df[cluster_cols])
        db = DBSCAN(eps=1.2, min_samples=3)
        df["cluster"] = db.fit_predict(X)

        plt.figure(figsize=(5, 4))
        plt.scatter(
            df["steps"],
            df["heart_rate"],
            c=df["cluster"],
            cmap="tab10",
            alpha=0.7
        )
        plt.xlabel("Steps")
        plt.ylabel("Heart Rate")
        plt.title("DBSCAN Activity Clusters")
        plt.tight_layout()

        cluster_path = f"reports/dbscan_{user_id}.png"
        plt.savefig(cluster_path)
        plt.close()

        story.append(Spacer(1, 16))
        story.append(Paragraph("<b>Activity Clustering (DBSCAN)</b>", styles["Heading2"]))
        story.append(Image(cluster_path, width=420, height=300))

    # ---------- BUILD PDF ----------
    doc = SimpleDocTemplate(pdf_path, pagesize=A4)
    doc.build(story)

    return FileResponse(
        pdf_path,
        media_type="application/pdf",
        filename=f"user_{user_id}_dashboard.pdf"
    )

# ================= DOWNLOAD ANOMALIES =================
@app.get("/download-anomalies")
def download_anomalies():
    if not os.path.exists("anomaly_report.csv"):
        return JSONResponse(status_code=404, content={"error": "No anomalies"})

    return FileResponse(
        "anomaly_report.csv",
        media_type="text/csv",
        filename="anomaly_report.csv"
    )
@app.get("/download-report")
def download_report():
    if CLEAN_DF is None:
        return JSONResponse(status_code=400, content={"error": "Run preprocess first"})

    filename = "fitpulse_dashboard_report.pdf"
    doc = SimpleDocTemplate(filename, pagesize=A4)
    styles = getSampleStyleSheet()
    story = []

    # ---------- TITLE ----------
    story.append(Paragraph("<b>FitPulse Health Analytics Report</b>", styles["Title"]))
    story.append(Spacer(1, 12))
    story.append(Paragraph(
        f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        styles["Normal"]
    ))
    story.append(Spacer(1, 20))

    # ---------- OVERVIEW ----------
    df = CLEAN_DF.copy()
    overview_data = [
        ["Metric", "Value"],
        ["Rows Loaded", len(df)],
        ["Users", df["user_id"].nunique()],
        ["Days", df["date"].nunique()],
        ["Avg Heart Rate", round(df["heart_rate"].mean(), 1)],
        ["Start Date", str(df["date"].min().date())],
        ["End Date", str(df["date"].max().date())],
    ]

    overview_table = Table(overview_data)
    overview_table.setStyle(TableStyle([
        ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ("GRID", (0,0), (-1,-1), 1, colors.black),
        ("FONT", (0,0), (-1,0), "Helvetica-Bold"),
    ]))

    story.append(Paragraph("<b>Dataset Overview</b>", styles["Heading2"]))
    story.append(Spacer(1, 10))
    story.append(overview_table)
    story.append(Spacer(1, 20))

    # ---------- SAMPLE DATA ----------
    story.append(Paragraph("<b>Sample Records</b>", styles["Heading2"]))
    sample_table = df_to_table(df)
    sample_table.setStyle(TableStyle([
        ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
        ("BACKGROUND", (0,0), (-1,0), colors.whitesmoke),
    ]))
    story.append(sample_table)
    story.append(Spacer(1, 20))

    # ---------- ANOMALY SUMMARY ----------
    if os.path.exists("anomaly_report.csv"):
        anom_df = pd.read_csv("anomaly_report.csv")
        story.append(Paragraph("<b>Anomaly Summary</b>", styles["Heading2"]))
        anom_table = df_to_table(anom_df)
        anom_table.setStyle(TableStyle([
            ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
            ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ]))
        story.append(anom_table)
        story.append(Spacer(1, 20))

    # ---------- IMAGES ----------
    story.append(Paragraph("<b>Visual Analytics</b>", styles["Heading2"]))
    story.append(Spacer(1, 10))

    image_files = (
      glob.glob("prophet_heart_rate_*.png") +
      glob.glob("prophet_steps_*.png") +
      glob.glob("prophet_sleep_*.png") +
      ["dbscan.png", "heart_rate_dist.png", "steps_dist.png", "sleep_dist.png"]
    )


    for img in image_files:
        if os.path.exists(img):
            story.append(Image(img, width=400, height=220))
            story.append(Spacer(1, 12))

    # ---------- BUILD ----------
    # ---------- RECOMMENDATIONS ----------
    if os.path.exists("recommendations.csv"):
        rec_df = pd.read_csv("recommendations.csv")
        story.append(Paragraph("<b>Health Recommendations</b>", styles["Heading2"]))
        rec_table = df_to_table(rec_df)
        rec_table.setStyle(TableStyle([
            ("GRID", (0,0), (-1,-1), 0.5, colors.grey),
            ("BACKGROUND", (0,0), (-1,0), colors.lightgrey),
        ]))
        story.append(rec_table)
        story.append(Spacer(1, 20))

    doc.build(story)

    return FileResponse(
        filename,
        media_type="application/pdf",
        filename=filename
    )




Writing backend.py


In [4]:
import threading
import uvicorn

def run_backend():
    uvicorn.run("backend:app", host="0.0.0.0", port=8003)

threading.Thread(target=run_backend).start()

In [5]:

backend_url = ngrok.connect(8003)
print("BACKEND URL:", backend_url)

BACKEND URL: NgrokTunnel: "https://renowned-zuri-coleopterous.ngrok-free.dev" -> "http://localhost:8003"


In [6]:
!mkdir -p .streamlit

In [7]:
%%writefile .streamlit/config.toml
[theme]
base="light"
primaryColor="#E63946"
backgroundColor="#F1FAEE"
secondaryBackgroundColor="#A8DADC"
textColor="#1D3557"

Writing .streamlit/config.toml


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import requests
import io
from datetime import datetime
import plotly.express as px

# ================= CONFIG =================
BACKEND = "http://localhost:8003"
HEADERS = {"ngrok-skip-browser-warning": "true"}

st.set_page_config(
    page_title="FitPulse Health Analytics",
    layout="wide"
)

# ================= SESSION STATE =================
if "preprocess_done" not in st.session_state:
    st.session_state.preprocess_done = False

if "module2_done" not in st.session_state:
    st.session_state.module2_done = False

# ================= API HELPERS =================
def api_get(endpoint, raw=False):
    r = requests.get(f"{BACKEND}{endpoint}", headers=HEADERS)

    if raw:
        return r

    try:
        return r.json()
    except Exception:
        st.error(f"Backend returned invalid JSON for {endpoint}")
        st.text(r.text[:500])  # show first 500 chars
        return {}


def api_post(endpoint, files=None):
    return requests.post(f"{BACKEND}{endpoint}", files=files, headers=HEADERS).json()

def api_post_json(endpoint, payload=None):
    r = requests.post(f"{BACKEND}{endpoint}", json=payload, headers=HEADERS)

    if r.status_code != 200:
        st.error(f"Backend error {r.status_code}")
        st.text(r.text)
        return {}

    try:
        return r.json()
    except Exception:
        st.error("Backend did not return JSON")
        st.text(r.text)
        return {}



# ================= COMMON FILTERS =================
def sidebar_filters(metrics=True):
    overview = api_get("/overview")
    users = ["All"]

    try:
        backend_users = overview.get("users_list", [])
        users += sorted([str(u) for u in backend_users])

    except:
        pass

    user = st.sidebar.selectbox("Select User", users)

    try:
        start = datetime.strptime(overview["start_date"], "%Y-%m-%d")
        end = datetime.strptime(overview["end_date"], "%Y-%m-%d")
    except:
        start, end = datetime.today(), datetime.today()

    dates = st.sidebar.date_input("Date Range", [start, end])

    metric = None
    if metrics:
        metric = st.sidebar.multiselect(
            "Select Metrics",
            ["heart_rate", "steps", "sleep"],
            default=["heart_rate", "steps", "sleep"]
        )

    return user, dates, metric

# ================= SIDEBAR =================
st.sidebar.title("FitPulse Controls")

page = st.sidebar.radio(
    "Navigation",
    [
        "1. Data Upload & Preprocessing",
        "2. Feature Extraction",
        "3. Trends",
        "4. Anomalies",
        "5. Distributions & DBSCAN",
        "6. User Dashboard",
        "7. Downloads"
    ]
)

st.sidebar.divider()
@st.cache_data(show_spinner=False)
def load_processed_data():
    res = api_get("/dataframe")
    if not res or "rows" not in res:
        return pd.DataFrame()
    return pd.DataFrame(res["rows"])

# ================= LOAD PROCESSED DATA =================
df = load_processed_data()
st.session_state.df = df


if not df.empty:
    df["user_id"] = df["user_id"].astype(str)
    df["date"] = pd.to_datetime(df["date"])

    # ---- column alignment with backend ----
    if "avg_heart_rate" in df.columns:
        df.rename(columns={"avg_heart_rate": "heart_rate"}, inplace=True)

    if "TotalSteps" in df.columns:
        df.rename(columns={"TotalSteps": "steps"}, inplace=True)

    if "total_sleep_minutes" in df.columns:
        df["sleep"] = df["total_sleep_minutes"] / 60


# ================= PAGE ROUTER =================
if page == "1. Data Upload & Preprocessing":
    st.title("FitPulse Health Analytics")
    with st.expander("Project Overview", expanded=True):
        st.markdown("""
**FitPulse** is an end-to-end health analytics system for wearable data.

**Input Columns**
- user_id
- date
- TotalSteps
- avg_heart_rate
- total_sleep_minutes

**Outputs**
- Cleaned dataset
- Rolling health features
- Anomalies & clustering
- Interactive trends
- CSV & PDF reports
        """)

    uploaded = st.file_uploader("Upload file", type=["csv", "json"])
    if st.button("Run Preprocessing"):
        if not uploaded:
            st.warning("Please upload a file first")
        else:
            res = api_post("/preprocess", files={"file": uploaded})
            if res.get("status") == "success":
                st.success("Preprocessing completed")
                st.session_state.preprocess_done = True

                ov = res["overview"]
                c1, c2, c3, c4 = st.columns(4)
                c1.metric("Records", ov["rows_loaded"])
                c2.metric("Users", ov["users"])
                c3.metric("Days", ov["days"])
                c4.metric("Avg HR", ov["avg_hr"])

                st.dataframe(pd.DataFrame(res["preview"]), use_container_width=True)
            else:
                st.error(res)

elif page == "2. Feature Extraction":

    st.title("Feature Extraction")


    if not st.session_state.preprocess_done:
        st.warning("Complete preprocessing first")
    else:
        # ---------- RUN MODULE2 ONCE ----------
        if not st.session_state.module2_done:
            res = api_post_json("/module2")
            if res.get("status") == "success":
                st.success("Feature extraction completed")
                st.session_state.module2_done = True
            else:
                st.error("Module2 failed. Check backend logs.")
                st.stop()


        # ---------- TSFRESH SECTION ----------
        st.subheader("TSFRESH Feature Insights (Full Dataset)")

        ts = api_get("/module3/tsfresh-summary")
        df_ts = pd.DataFrame(ts.get("features", []))

        if df_ts.empty:
            st.info("No TSFRESH features available")
        else:
            st.success("TSFRESH feature extraction completed")

            # Chart
            fig = px.bar(
                df_ts,
                x="description",
                y="importance",
                title="Key Behavioral Patterns (All Users)",
                labels={"description": "Behavioral Insight", "importance": "Impact Level (log scale)"}
            )
            fig.update_layout(xaxis_tickangle=-30)
            st.plotly_chart(fig, use_container_width=True)

            # Table
            st.dataframe(
                df_ts.rename(columns={"description": "Behavioral Insight", "importance": "Impact Level"}),
                use_container_width=True
            )






elif page == "3. Trends":
    if not st.session_state.module2_done:
        st.warning("Complete previous modules first")
    else:
        st.title("Health Trends")
        user, _, metrics = sidebar_filters(metrics=True)

        if not metrics:
            st.warning("Select at least one metric")
        else:
            for metric in metrics:
                r = api_get(f"/module3/prophet/{metric}?user_id={user}", raw=True)
                if r.status_code == 200:
                    st.image(io.BytesIO(r.content), use_column_width=True)
                else:
                    st.error(r.json().get("error", "No data"))

elif page == "4. Anomalies":
    if not st.session_state.module2_done:
        st.warning("Complete previous modules first")
    else:
        st.title("Anomaly Analysis")
        user, _, metrics = sidebar_filters(metrics=True)

        # ------------------ Anomaly Summary ------------------
        summary = api_get(f"/module3/summary?user_id={user}")
        summary_dict = summary.get("summary", {}) if isinstance(summary, dict) else {}

        if not summary_dict:
            st.info("No anomalies found for selected user.")
        else:
            df_summary = pd.DataFrame(
                summary_dict.items(),
                columns=["Metric", "Count"]
            )

            metric_map = {
                "heart_rate": ["heart_rate_high", "heart_rate_low"],
                "steps": ["no_steps"],
                "sleep": ["sleep_low", "sleep_high"]
            }

            selected_metrics = []
            if metrics:
                for m in metrics:
                    selected_metrics.extend(metric_map.get(m, []))

            if selected_metrics:
                df_summary_plot = df_summary[df_summary["Metric"].isin(selected_metrics)]
            else:
                df_summary_plot = df_summary.copy()

            if df_summary_plot.empty:
                st.info("No anomalies found for selected metric filters.")
            else:
                fig = px.bar(
                    df_summary_plot,
                    x="Metric",
                    y="Count",
                    color="Metric",
                    title="Anomaly Summary"
                )
                st.plotly_chart(fig, use_container_width=True)

        # ------------------ Detailed Health Insights ------------------
        st.subheader("Detailed Health Insights")
        combined_table = api_get(
            f"/module3/anomaly-with-recommendations?user_id={user}"
        )

        rows = combined_table.get("rows", []) if isinstance(combined_table, dict) else []

        if rows:
            df_combined = pd.DataFrame(rows)

            # ✅ FIX: ensure user_id exists
            if "user_id" not in df_combined.columns:
                df_combined["user_id"] = user

            # Sort by severity
            if "severity" in df_combined.columns:
                severity_map = {"High": 3, "Medium": 2, "Low": 1}
                df_combined["severity_rank"] = df_combined["severity"].map(severity_map)
                df_combined = df_combined.sort_values(
                    by="severity_rank",
                    ascending=False
                )

            display_cols = ["user_id", "date", "issue", "severity", "recommendation"]

            st.dataframe(
                df_combined[display_cols],
                use_container_width=True,
                height=380
            )

            # Download CSV
            csv = df_combined[display_cols].to_csv(index=False).encode("utf-8")

            st.download_button(
                "Download Detailed Insights",
                csv,
                f"user_{user}_health_insights.csv"
            )
        else:
            st.info("No detailed health insights available for this user.")



elif page == "5. Distributions & DBSCAN":
    if not st.session_state.module2_done:
        st.warning("Complete previous modules first")
    else:
        st.title("Distributions & Clustering")
        user, _, metrics = sidebar_filters(metrics=True)

        for metric in metrics:
            r = api_get(f"/module3/distribution/{metric}?user_id={user}", raw=True)
            if r.status_code == 200:
                st.image(io.BytesIO(r.content), caption=f"{metric} distribution", use_column_width=True)

        if any(m in metrics for m in ["heart_rate", "steps", "sleep"]):
            r = api_get(f"/module3/dbscan?user_id={user}", raw=True)
            if r.status_code == 200:
                st.image(io.BytesIO(r.content), caption="DBSCAN clustering", use_column_width=True)

# ================= PAGE 6 : USER HEALTH DASHBOARD =================
elif page == "6. User Dashboard":
    st.title("User Health Dashboard")

    # ---------------- LOAD USER DATA ----------------
    df = st.session_state.get("df", pd.DataFrame())
    if df.empty:
        res = api_get("/dataframe")
        if "rows" in res and res["rows"]:
            df = pd.DataFrame(res["rows"])
            df["date"] = pd.to_datetime(df["date"])
            st.session_state.df = df
        else:
            st.warning("No processed data available. Run preprocessing first.")
            st.stop()

    # ---------------- SELECT USER ----------------
    user_ids = df["user_id"].unique() if "user_id" in df.columns else ["Demo User"]
    selected_user = st.selectbox("Select User", user_ids)

    # ✅ THIS LINE FIXES EVERYTHING
    st.session_state.selected_user = selected_user

    user_data = df[df["user_id"] == selected_user].copy()


    if user_data.empty:
        st.warning("No data available for this user.")
        st.stop()

    # ---------------- MERGE ANOMALIES ----------------
    anom = api_get(f"/module3/anomaly-with-recommendations?user_id={selected_user}")
    anom_rows = anom.get("rows", [])
    if anom_rows:
        df_anom = pd.DataFrame(anom_rows)
        df_anom["date"] = pd.to_datetime(df_anom["date"])
        user_data = user_data.merge(
            df_anom[["date", "issue", "severity", "recommendation"]],
            on="date",
            how="left"
        )
    else:
        user_data["issue"] = None
        user_data["severity"] = None
        user_data["recommendation"] = None

    # ---------------- ROW 1: Key Metrics ----------------
    avg_heart = round(user_data["heart_rate"].mean(), 1)
    avg_steps = round(user_data["steps"].mean(), 0)
    avg_sleep = round(user_data["sleep"].mean(), 1)

    # ---------------- HEALTH SCORE (Continuous) ----------------
    health_data = api_get(f"/module3/health-score?user_id={selected_user}")

    if health_data:
        health_score = health_data.get("health_score", 0)
        hr_score = health_data["components"]["heart_rate"]
        steps_score = health_data["components"]["steps"]
        sleep_score = health_data["components"]["sleep"]
    else:
        health_score = 0
        hr_score = steps_score = sleep_score = 0

    col1, col2, col3, col4 = st.columns(4)
    col1.metric("Avg Heart Rate (BPM)", avg_heart)
    col2.metric("Avg Steps", avg_steps)
    col3.metric("Avg Sleep (hrs)", avg_sleep)
    col4.metric("Health Score (%)", health_score)

    # ---------------- ROW 2: Health Status & Table Side by Side ----------------
    # ---------------- ROW 2: Health Status & Table Side by Side ----------------
    st.subheader("Health Status & User Issues")
    col1, col2 = st.columns(2)

    # --- Donut chart for Health Status ---
    import plotly.graph_objects as go

    fig_donut = go.Figure(data=[go.Pie(
        labels=["Heart Rate", "Steps", "Sleep"],
        values=[hr_score, steps_score, sleep_score],
        hole=0.6,
        textinfo="label+percent",
        marker=dict(colors=["crimson", "royalblue", "green"])
    )])

    fig_donut.update_layout(
        title=f"{selected_user} Health Score",
        annotations=[dict(
            text=f"{health_score}%",
            x=0.5, y=0.5,
            font_size=22,
            showarrow=False
        )],
        margin=dict(t=50, b=20, l=20, r=20)
    )

    # 👈 THIS was missing
    col1.plotly_chart(fig_donut, use_container_width=True)

    # --- Table for user data ---
    display_cols = ["date", "heart_rate", "steps", "sleep"]
    if "issue" in user_data.columns:
        display_cols += ["issue", "severity", "recommendation"]

    col2.dataframe(
        user_data[display_cols].sort_values("date", ascending=False),
        use_container_width=True
    )



    # ---------------- ROW 3: Raw vs Rolling Metrics ----------------
    st.subheader("Trends: Raw vs Rolling Average")
    metrics = ["heart_rate", "steps", "sleep"]
    for metric in metrics:
        if metric in user_data.columns:
            user_data[f"{metric}_rolling"] = user_data[metric].rolling(3, min_periods=1).mean()
            fig = px.line(user_data, x="date", y=[metric, f"{metric}_rolling"],
                          labels={"value": metric, "variable": "Type"},
                          title=f"{metric.replace('_',' ').title()} (Raw vs Rolling)")
            st.plotly_chart(fig, use_container_width=True)

    # ---------------- ROW 4: DBSCAN Clusters ----------------
    st.subheader("Activity Clusters (DBSCAN)")
    from sklearn.preprocessing import StandardScaler
    from sklearn.cluster import DBSCAN

    cluster_cols = ["heart_rate", "steps", "sleep"]
    if all(col in user_data.columns for col in cluster_cols):
        X = StandardScaler().fit_transform(user_data[cluster_cols])
        db = DBSCAN(eps=1.5, min_samples=2).fit(X)
        user_data["cluster"] = db.labels_
        fig = px.scatter_3d(user_data, x="heart_rate", y="steps", z="sleep",
                            color="cluster", title="DBSCAN Clusters")
        st.plotly_chart(fig, use_container_width=True)



    # =================================================
    # USER-SPECIFIC REPORT GENERATION
    # =================================================

elif page == "7. Downloads":
    st.title("Download Reports")

    # ---------- FULL DASHBOARD PDF ----------
    st.subheader("Full Dashboard PDF")

    st.markdown("""
    This PDF contains:
    - Dataset Overview
    - Sample Records
    - Anomaly Summary
    - Health Metrics
    - Visual Analytics
    - Recommendations
    """)

    if st.button("Generate & Download Full Dashboard PDF"):
        r = requests.get(f"{BACKEND}/download-report", headers=HEADERS)
        if r.status_code == 200:
            st.download_button(
                "Download Full Dashboard PDF",
                r.content,
                file_name="fitpulse_dashboard_report.pdf",
                mime="application/pdf"
            )
        else:
            st.error("Failed to generate full dashboard PDF")

    # ---------- PERSONALIZED USER PDF ----------
    st.subheader("Personalized User Report")

    selected_user = st.session_state.get("selected_user")

    if not selected_user:
        st.warning("Please select a user from the Dashboard page first.")
    else:
        if st.button("Generate User Dashboard PDF"):
            r = requests.get(
                f"{BACKEND}/module4/user-dashboard-report?user_id={selected_user}",
                headers=HEADERS
            )

            if r.status_code == 200:
                st.download_button(
                    "Download User PDF",
                    r.content,
                    file_name=f"user_{selected_user}_dashboard.pdf",
                    mime="application/pdf"
                )
            else:
                st.error("Failed to generate personalized report")






st.caption("FitPulse | FastAPI Backend with Streamlit Frontend")

Writing app.py


In [9]:
!streamlit run app.py --server.port 8501 --server.address 0.0.0.0




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501



  Stopping...


In [10]:
# ===============================
# Launch Streamlit via Ngrok
# ===============================
import threading, time
def run_streamlit():
    !streamlit run app.py --server.port 8501 --server.address 0.0.0.0
threading.Thread(target=run_streamlit).start()
time.sleep(5)
streamlit_url = ngrok.connect(8501)
print("Streamlit public URL:", streamlit_url)




  You can now view your Streamlit app in your browser.

  URL: http://0.0.0.0:8501

Streamlit public URL: NgrokTunnel: "https://renowned-zuri-coleopterous.ngrok-free.dev" -> "http://localhost:8501"
